In [ ]:
import os 
os.chdir("/home/canyon/Bus-Weather-Impacts")
from src.utils import *
import pandas as pd
import os
import osmnx as ox
import geopandas as gpd
import networkx as nx
from sklearn.neighbors import KDTree
import numpy as np
from plotnine import *

In [ ]:
bus_raw = pd.read_parquet("/home/data/bus-weather/raw_bus_gtfs_rt_202230917_20230930.parquet")

In [ ]:
bus_gdf = gpd.GeoDataFrame(bus_raw, geometry=gpd.points_from_xy(bus_raw['vehicle.position.longitude'], bus_raw['vehicle.position.latitude'])).set_crs(4326)

In [ ]:
bus_gdf['date'] = pd.to_datetime(bus_gdf['vehicle.timestamp']).dt.date
bus_gdf['hour'] = pd.to_datetime(bus_gdf['vehicle.timestamp']).dt.hour

In [ ]:
bus_gdf_trim = bus_gdf[["vehicle.trip.trip_id", "vehicle.vehicle.id", "date", "hour", "geometry"]]

In [ ]:
ntas = gpd.read_file("https://data.cityofnewyork.us/resource/9nt8-h7nd.geojson")
ntas = ntas[["ntaname", "boroname", "geometry"]]

In [ ]:
bus_nta = gpd.sjoin(bus_gdf_trim, ntas, how="inner", op="within")

In [ ]:
bus_nta

In [ ]:
bus_counts = bus_nta.groupby(["date", "hour", "ntaname", "boroname"]).agg({"vehicle.trip.trip_id" : "nunique"}).reset_index().rename({"vehicle.trip.trip_id" : "unique_bus_count"}, axis = 1)

In [ ]:
bus_counts['combined_datetime'] = pd.to_datetime(bus_counts['date']) + pd.to_timedelta(bus_counts['hour'], unit='h')

In [ ]:
bus_counts.ntaname.drop_duplicates().sample(30)

In [ ]:
ggplot(bus_counts.query("ntaname.isin(['East Williamsburg'])"), aes(y = "unique_bus_count", x = "combined_datetime", color = "ntaname", group = "ntaname")) + geom_line()